In [1]:
import sys
sys.path.insert(0, '/home/yw440/MXGPY/mxgpy')
sys.path.insert(0, '/home/yw440/MyRGPs/')

import os
os.environ['MXNET_ENGINE_TYPE'] = 'NaiveEngine'

import scipy.io
from pylab import *
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import GPy
import numpy as np
np.random.seed(101)

from mxgpy.rgp.rgp import RGP 
from mxgpy.kernel.rbf import RBF
import mxnet as mx
ctx = mx.gpu()
mx.Context.default_ctx = ctx

from mxgpy.util import positive_transform_reverse_numpy
import helper

 /anaconda/envs/py35/lib/python3.5/site-packages/urllib3/contrib/pyopenssl.py:46: DeprecationWarning:OpenSSL.rand is deprecated - you should use os.urandom instead
 /anaconda/envs/py35/lib/python3.5/site-packages/mxnet/context.py:144: DeprecationWarning:Context.default_ctx has been deprecated. Please use Context.current_context() instead. Please use test_utils.set_default_context to set a default context


In [2]:
def compute_delta(data_list):
#data_list = y_train;
    delta_list = [];
    for bone in data_list:
        delta_bone = [];
        for seq in bone:
            delta_seq = np.asarray([seq[i,:]-seq[i-1,:] for i in range(1,len(seq))])
            delta_seq = np.vstack((delta_seq, np.zeros(seq.shape[-1])))
            delta_bone.append(delta_seq)
    
        delta_list.append(delta_bone)
    return delta_list

In [3]:
train_motions = ['01', '02', '03', '04']
test_motions = ['05', '06', '07', '08']
subject = '35'
y_bones = ['root', 'lfemur', 'ltibia', 'lfoot', 'ltoes', 'rfemur', 'rtibia', 'rfoot', 'rtoes', 
             'lowerback', 'upperback', 'thorax', 'lowerneck', 'upperneck', 'head', 
             'lclavicle', 'lhumerus', 'lradius', 'lwrist', 'lhand', 'lfingers', 'lthumb',
            'rclavicle', 'rhumerus', 'rradius', 'rwrist', 'rhand', 'rfingers', 'rthumb']
# list of control signal in order
u_bones = ['None', 'root', 'lfemur', 'ltibia', 'lfoot', 'root', 'rfemur', 'rtibia', 'rfoot', 
      'root', 'lowerback', 'upperback', 'thorax', 'lowerneck', 'upperneck',
      'thorax', 'lclavicle', 'lhumerus', 'lradius', 'lwrist', 'lhand', 'lwrist',
      'thorax', 'rclavicle', 'rhumerus', 'rradius', 'rwrist', 'rhand', 'rwrist' ]

m1 = GPy.util.datasets.cmu_mocap(subject, train_motions, test_motions, sample_every=4, data_set='cmu_mocap')
y_train = helper.extract_independent_bones(m1['Y'], m1['lbls']) # list of lists of arrays
y_test = helper.extract_independent_bones(m1['Ytest'], m1['lblstest'])
y_train_delta = compute_delta(y_train)
y_test_delta = compute_delta(y_test)
print(np.shape(y_train_delta), np.shape(y_test_delta))
   
y_train_dict = {}
y_train_mean_list, y_train_std_list = [], []
for i, item in enumerate(y_train):
    if i==0:
        out, mean, std = helper.normalise_data(y_train_delta[0])
        y_train_dict[y_bones[0]] = out
        y_train_mean_list.append(mean)
        y_train_std_list.append(std)
    else:
        out, mean, std = helper.normalise_data(item)
        y_train_dict[y_bones[i]] = out
        y_train_mean_list.append(mean)
        y_train_std_list.append(std)
        
y_test_dict = {}
y_test_mean_list, y_test_std_list = [], []
for i, item in enumerate(y_test):
    if i==0:
        out, mean, std = helper.normalise_data(y_test_delta[0])
        y_test_dict[y_bones[0]] = out
        y_test_mean_list.append(mean)
        y_test_std_list.append(std)
    else:
        out, mean, std = helper.normalise_data(item)
        y_test_dict[y_bones[i]] = out
        y_test_mean_list.append(mean)
        y_test_std_list.append(std)
    

(29, 4) (29, 4)


In [ ]:
## TRAINING
for j, bone in enumerate(y_bones):  
    print(str(j)+'-'+bone)
    y_train = y_train_dict[bone]
    if bone is 'root':
        u_train = [np.ones((np.shape(y_train[0])[0],1)),
                   np.ones((np.shape(y_train[1])[0],1)),
                   np.ones((np.shape(y_train[2])[0],1)),
                   np.ones((np.shape(y_train[3])[0],1))]
    else:
        u_train = y_train_dict[u_bones[j]]
        
    YD = np.shape(y_train[0])[1] # dimension of the data, 62 dimension
    UD = np.shape(u_train[0])[1] # dimension of the control signal
    ## Initialise model
    win_out = 20 # only consider the feature from previous time step
    win_in = 20
    m_control = RGP(wins=[win_out], with_control=True, X_dims=[YD], U_dim=UD, U_win=win_in, num_inducing=100,
                    kernels=[RBF(win_out*YD+win_in*UD, ARD=True)], ctx=ctx, dtype=np.float64)
    
    ## Initialize parameters
    m_control.collect_params()['rbf_variance'].set_data(positive_transform_reverse_numpy(np.array([3.]), np.float64))
    m_control.collect_params()['rbf_lengthscale'].set_data(positive_transform_reverse_numpy(np.ones((win_out*YD+win_in*UD,))*5,np.float64))
    m_control.collect_params()['layer_0_noise_var'].set_data(positive_transform_reverse_numpy(np.array([0.0001]),np.float64))

    ## Train model
    m_control.optimize(Y=y_train, U=u_train, max_iters=500, step_rate=0.2)
    m_control.collect_params()['layer_0_noise_var'].grad_req = 'null' # fix the noise variance to a single value
    m_control.optimize(Y=y_train, U=u_train, max_iters=500, step_rate=0.1)
    m_control.collect_params()['layer_0_noise_var'].grad_req = 'write' # write new noise variance to a single value
    m_control.optimize(Y=y_train, U=u_train, max_iters=1000, step_rate=0.05)

    m_control.save_params('./models/delta_sequence_root/'+str(bone))

0-root
31396530.194532074
22950454.869404614
17096882.437514074
12676340.292643422
9218533.003272947
6539294.52740331
4551850.01662764
3165123.1040794193
2249550.1398973837
1660446.351493592
1276870.9250909109
1017144.8010844358
832509.1280827511
695212.1169315114
589341.4362018006
505474.97365874995
437796.26176297374
382502.6083904904
336934.1209325575
299109.63800885464
267493.01960390277
240876.71391642722
218310.97040932858
199049.53200276507
182505.18215658545
168213.49925867404
155803.9608668778
144977.6163688107
135490.26315465698
127140.12333016696
119759.1294883874
113206.80673656285
107365.68941433224
102137.50122847513
97439.81230836664
93203.17397236249
89368.77820484698
85886.62551918367
82714.1086413285
79814.90777875917
77158.08183994415
74717.25993801534
72469.8642180195
70396.36299442136
68479.59618338305
66704.23095151721
65056.38980982561
63523.45319377689
62094.000277267754
60757.82086358864
59505.921344353716
58330.467711198595
57224.65661320755
56182.549764926705

15031.916633041255
15017.60858466175
15003.360199020472
14989.170925855375
14975.040221174246
14960.967545469735
14946.952364587098
14932.994149702197
14919.092377299541
14905.246529150212
14891.45609229318
14877.720558163446
14864.039424278053
14850.41219251689
14836.83836995118
14823.317469660344
14809.849009029884
14796.432511400897
14783.067503565886
14769.753518235822
14756.490093995344
14743.276772845235
14730.113102621202
14716.998635362515
14703.932928892775
14690.915545995733
14677.946053599397
14665.024023558068
14652.14903341568
14639.320664807537
14626.538504229182
14613.802143017925
14601.111177327566
14588.465208109887
14575.863840325117
14563.306685214478
14550.793357224937
14538.323475514197
14525.896665433957
14513.512554736395
14501.170777314039
14488.870971657952
14476.61277936665
14464.39584807037
14452.219829188529
14440.084378639203
14427.989156808453
14415.93382779476
14403.918061545044
14391.94153092855
14380.003913869215
14368.104891866942
14356.244151390285
14

9479.14391310311
9478.250991904588
9474.176918622466
9468.917807085423
9463.586828083016
9461.467795517337
9462.380268217505
9465.075676392318
9467.394501460476
9464.465697684762
9454.202926413127
9443.00042033441
9443.831185092467
9448.751550440182
9441.811832223473
9435.289474367539
9434.404898368579
9433.719070037681
9434.187539192197
9436.152301494534
9436.571911142739
9424.895781594336
9414.179596071726
9419.567209816083
9420.150425498246
9409.310745901155
9402.04157203849
9402.41595974048
9407.065872239848
9409.318568807827
9403.294484184082
9400.078830766879
9403.115217331713
9398.873513177801
9389.263122099583
9391.120594054533
9389.308512277019
9379.952842144208
9378.657234142529
9376.446029813225
9372.642400840658
9368.614420649597
9369.643340877414
9369.435461870464
9368.557396726015
9369.068797180264
9369.771556203046
9363.46859963331
9354.27575930494
9352.994693158154
9354.543988788413
9353.456244153127
9350.587960372486
9346.908214065345
9339.609463887868
9334.09381830254

1977.8169415674683
1977.755028783584
1977.6935796090684
1977.6325907056141
1977.572060902208
1977.511989015039
1977.45237217687
1977.3932046742566
1977.3344772899736
1977.276177091303
1977.2182875463077
1977.160788942584
1977.1036590044814
1977.0468736303208
1976.9904077044748
1976.9342358973468
1976.8783334130064
1976.8226765862446
1976.7672433375142
1976.712013431791
1976.6569685517745
1976.602092186562
1976.5473694105176
1976.4927865582447
1976.4383308787333
1976.3839901873703
1976.3297525572214
1976.275606092786
1976.2215387643928
1976.1675382971657
1976.1135921554924
1976.0596875325436
1976.005811396527
1975.951950548164
1975.898091686487
1975.8442214752422
1975.7903266040807
1975.736393836551
1975.6824100561985
1975.6283622830065
1975.574237688441
1975.520023576888
1975.4657073756512
1975.4112766010444
1975.3567188196275
1975.3020216246302
1975.2471725881298
1975.1921592584795
1975.1369691357827
1975.0815896879494
1975.0260083517794
1974.9702125536187
1974.9141897439167
1974.8579

1963.2015626336856
1963.1718260922669
1963.1414944439462
1963.1105298459286
1963.0788925218353
1963.046540854046
1963.0134315716118
1962.9795200258764
1962.944760599197
1962.9091072808474
1962.8725144561838
1962.8349379701226
1962.7963365154196
1962.7566734681063
1962.7159192292888
1962.674054230349
1962.6310726915576
1962.586987321186
1962.5418350795173
1962.495684161351
1962.4486422517932
1962.4008660067038
1962.3525715289593
1962.3040451101454
1962.2556529193662
1962.2078471140107
1962.1611643151195
1962.1162099989424
1962.073619872704
1962.0339877168674
1961.997752001315
1961.9650468458706
1961.9355535152267
1961.9084320604263
1961.8824344030852
1961.8562384496981
1961.8288868615452
1961.8000906836787
1961.7702241023649
1961.7400480698743
1961.7103514050204
1961.6816857891258
1961.6542663072346
1961.628014000538
1961.6026736346694
1961.5779413539244
1961.5535602312013
1961.5293680416416
1961.5053009417254
1961.4813671010054
1961.4576075411564
1961.4340596311777
1961.410733819871
19

...

17084.35074849544
16991.64947108139
16900.481855836355
16810.812119842805
16722.6100873338
16635.83994905783
16550.462827271374
16466.44341667542
16383.746506896083
16302.331900143075
16222.161403501119
16143.208302640549
16065.452756118582
15988.870304434753
15913.43126196541
15839.108059270804
15765.877380585289
15693.71836560016
15622.611904713121
15552.53703932013
15483.468315824735
15415.382442911403
15348.263352247805
15282.094566616071
15216.854437243615
15152.523438852837
15089.086337492718
15026.526500422344
14964.82648169513
14903.969758110972
14843.938321468668
14784.716281337078
14726.291816428271
14668.6504446422
14611.776216059316
14555.655630208821
14500.27301502947
14445.613861963391
14391.66819798158
14338.42193198206
14285.859769752758
14233.97072663089
14182.741437875406
14132.158836269049
14082.21203129246
14032.888162513345
13984.176232837723
13936.065004234151
13888.542107557827
13841.597081803273
13795.218919824056
13749.397561257965
13704.122281658445
13659.3820

7066.583156037328
7040.934593973892
7016.167679746133
6992.2322249333465
6968.903533740489
6946.455723194899
6925.031050375408
6904.428580703207
6884.456891035084
6864.953032768868
6845.833698740875
6827.093579896507
6808.768351139854
6791.006876433565
6773.763636607165
6756.875395540772
6740.25990906585
6723.943749694021
6707.980832060312
6692.328606696174
6677.034448373128
6662.081394668676
6647.406965914828
6633.165071774796
6619.749265973374
6608.131606470526
6596.190454019378
6580.620047548764
6564.073710508647
6553.311148144685
6540.989448215798
6525.28845152939
6513.734129645834
6502.209452431342
6487.869267276971
6475.7925996434515
6464.588479240339
6451.855398569783
6439.363791150476
6427.768244027249
6416.062726756506
6404.071086943064
6392.526094649105
6381.756590818588
6370.805459086754
6359.049085694957
6347.097512826549
6335.812441622288
6325.173249272036
6314.844151233713
6304.525555338642
6294.191986350885
6283.956409239384
6273.942885442179
6264.361866118494
6256.60033

6126.271599456048
5225.53553834329
4939.404878405474
4814.26243663193
4717.694445249724
4609.069503242115
4482.394905237841
4352.241224874312
4229.517866233898
4116.820770457902
4011.145518497332
3910.528113923395
3814.2003300727265
3720.5907801450876
3628.811685099129
3539.5000642944055
3453.6359043996476
3371.4753800097624
3292.7265375695597
3217.098453765502
3144.3989034459346
3074.4239634740484
3007.039652015983
2942.2871559116466
2880.2279294134437
2820.7466249839295
2763.613821724867
2708.672384107671
2655.877083487792
2605.183680188342
2556.462813408344
2509.5352683743386
2464.263610286906
2420.590875441061
2378.502143738556
2337.967684674353
2298.9245758845673
2261.296224256449
2225.0137696140473
2190.018294695151
2156.252962793635
2123.6619056908303
2092.197055349836
2061.820684014921
2032.4976956750043
2004.1857309847785
1976.8335128711933
1950.3882472150253
1924.8044827544325
1900.0473153618896
1876.0888880338684
1852.902306780422
1830.457835381649
1808.7233003926003
1787.66

920.7951674468538
920.6681320870596
920.541513622894
920.4152962981666
920.2894639518647
920.1640000206405
920.0388875182948
919.9141090465105
919.7896467674677
919.6654824229666
919.5415973309296
919.4179723598131
919.2945879493816
919.1714240943408
919.0484603592218
918.9256758716995
918.8030493173309
918.6805589507622
918.5581826085527
918.4358977015398
918.3136812317689
918.1915098043653
918.069359642149
917.9472066032783
917.825026194891
917.7027936201275
917.580483777529
917.4580713094526
917.3355306556025
917.2128360604086
917.0899616605398
916.9668815274704
916.8435697173788
916.7200003736438
916.596147786297
916.47198649068
916.3474913690939
916.2226377561774
916.0974015863201
915.9717595016841
915.8456890265446
915.7191687190038
915.5921783686231
915.4646991882752
915.3367140159277
915.208207580767
915.0791667510101
914.9495807633263
914.8194416040842
914.6887442508869
914.5574870439301
914.4256720649079
914.2933054917166
914.1603980378782
914.0269653509276
913.8930284779929


902.6191048781113
902.6138286214418
902.6085636315973
902.6033098428738
902.5980671902016
902.5928356086202
902.5876150327638
902.582405397852
902.5772066386726
902.5720186905692
902.5668414889255
902.5616749686579
902.5565190651959
902.5513737139731
902.5462388499225
902.5411144079634
902.5360003234796
902.5308965318159
902.5258029677832
902.5207195666238
902.5156462630324
902.5105829921174
902.5055296884261
902.5004862869047
902.4954527219279
902.4904289282558
902.4854148400686
902.4804103919206
902.4754155177804
902.4704301515062
902.4654542273199
902.460487679324
902.455530441029


In [4]:
def test_run(model, seq, win_out, win_in, test_data, control_signal, path, MEAN_PRED=True, with_control=True):
    if with_control:
        YD = np.shape(test_data[0])[-1]; print('Observationa dimension:', YD)
        UD = np.shape(control_signal[0])[-1] ; print('Control signal dimension:', UD)
    
        m = RGP(wins=[win_out], with_control=True, X_dims=[YD], U_dim=UD, U_win=win_in, 
            num_inducing=100, kernels=[RBF(win_out*YD+win_in*UD, ARD=True)], 
            ctx=ctx, dtype=np.float64)
        #'./models/fully_observed/'
        m.load_params(path+model)
    
        y_pd = m.layers[0].freerun(test_data[seq].shape[0]-win_out, init_X=test_data[seq][:win_out,:],
                               U=control_signal[seq], mean_predict=False, nSamples=100)
    
        y_pd_mean = y_pd.mean(0) # the first dimention of the matrix is number of samples
        y_pd_std = y_pd.std(0)
    
    if MEAN_PRED:
        return y_pd_mean
    else:
        return y_pd_mean + y_pd_std*np.random.normal(0,0.5,size=(y_pd_mean.shape[0], y_pd_mean.shape[1]))

In [5]:
y_test_list = []
for j, bone in enumerate(y_bones):  
    y_test_list.append(y_test_dict[bone])
    
u_test_list = [np.ones((np.shape(y_test[0][0])[0],1)), 
               np.ones((np.shape(y_test[0][1])[0],1)),
               np.ones((np.shape(y_test[0][2])[0],1)), 
               np.ones((np.shape(y_test[0][3])[0],1))]

## collect predictions from different dimensions and concatenate into a large matrix
## This is the correct order
bones = ['root', 'lfemur', 'ltibia', 'lfoot', 'ltoes', 'rfemur', 'rtibia', 'rfoot', 'rtoes', 
        'lowerback', 'upperback', 'thorax', 'lowerneck', 'upperneck', 'head', 
        'lclavicle', 'lhumerus', 'lradius', 'lwrist', 'lhand', 'lfingers', 'lthumb',
        'rclavicle', 'rhumerus', 'rradius', 'rwrist', 'rhand', 'rfingers', 'rthumb']
u_bones = ['None', 'root', 'lfemur', 'ltibia', 'lfoot', 'root', 'rfemur', 'rtibia', 'rfoot', 
          'root', 'lowerback', 'upperback', 'thorax', 'lowerneck', 'upperneck',
          'thorax', 'lclavicle', 'lhumerus', 'lradius', 'lwrist', 'lhand', 'lwrist',
          'thorax', 'rclavicle', 'rhumerus', 'rradius', 'rwrist', 'rhand', 'rwrist' ]

win_out = 20
win_in = 20
control_signal_dict = {}
num_train_seq = np.shape(y_test_list)[1]
y_pds = [[],[],[],[]]
for i, model in enumerate(bones):
    print ("="*10 + model + "="*10)
    if i==0:
        u_test = u_test_list
    else:
        u_test = control_signal_dict[u_bones[i]]
    u_test_list = []
    for seq in range(num_train_seq):
        y_pd_temp = test_run(model, seq, win_out, win_in, y_test_list[i], u_test, path="./models/delta_sequence_root/", MEAN_PRED=True, with_control=True) # output array
        u_test_list.append(y_pd_temp)
        if i==0:
            y_pds[seq] = y_pd_temp
        else:
            y_pds[seq] = np.hstack((y_pds[seq], y_pd_temp))
    control_signal_dict[model] = u_test_list
    #print(control_signal_dict)



==========root==========
Observationa dimension: 6
Control signal dimension: 1
Observationa dimension: 6
Control signal dimension: 1
Observationa dimension: 6
Control signal dimension: 1
Observationa dimension: 6
Control signal dimension: 1
==========lfemur==========
Observationa dimension: 3
Control signal dimension: 6
Observationa dimension: 3
Control signal dimension: 6
Observationa dimension: 3
Control signal dimension: 6
Observationa dimension: 3
Control signal dimension: 6
==========ltibia==========
Observationa dimension: 1
Control signal dimension: 3
Observationa dimension: 1
Control signal dimension: 3
Observationa dimension: 1
Control signal dimension: 3
Observationa dimension: 1
Control signal dimension: 3
==========lfoot==========
Observationa dimension: 2
Control signal dimension: 1
Observationa dimension: 2
Control signal dimension: 1
Observationa dimension: 2
Control signal dimension: 1
Observationa dimension: 2
Control signal dimension: 1
==========ltoes==========
Obser

 /home/yw440/MXRGP/mxgpy/util.py:182: RuntimeWarning:invalid value encountered in greater


Observationa dimension: 1
Control signal dimension: 2
Observationa dimension: 1
Control signal dimension: 2
Observationa dimension: 1
Control signal dimension: 2
==========lthumb==========
Observationa dimension: 2
Control signal dimension: 1
Observationa dimension: 2
Control signal dimension: 1
Observationa dimension: 2
Control signal dimension: 1
Observationa dimension: 2
Control signal dimension: 1
==========rclavicle==========
Observationa dimension: 2
Control signal dimension: 3
Observationa dimension: 2
Control signal dimension: 3
Observationa dimension: 2
Control signal dimension: 3
Observationa dimension: 2
Control signal dimension: 3
==========rhumerus==========
Observationa dimension: 3
Control signal dimension: 2
Observationa dimension: 3
Control signal dimension: 2
Observationa dimension: 3
Control signal dimension: 2
Observationa dimension: 3
Control signal dimension: 2
==========rradius==========
Observationa dimension: 1
Control signal dimension: 3
Observationa dimension

In [8]:
print(helper.compute_MSE(y_test_list, y_pds))

[nan, nan, nan, nan]


In [6]:
def pred_plot(seq, win_out, win_in, model, data_in, control_signal, dim,path, steps=None, WRITE_FIG=False):
    YD = np.shape(data_in[0])[-1]; print('Observationa dimension:', YD)
    UD = np.shape(control_signal[0])[-1] ; print('Control signal dimension:', UD)
    
    m = RGP(wins=[win_out], with_control=True, X_dims=[YD], U_dim=UD, U_win=win_in, 
            num_inducing=100, kernels=[RBF(win_out*YD+win_in*UD, ARD=True)], ctx=ctx, dtype=np.float64)
    m.load_params(path+model)
    #data_in[seq].shape[0]-win_out
    if steps is None:
        steps = data_in[seq].shape[0]-win_out
    y_pd = m.layers[0].freerun(steps, init_X=data_in[seq][:win_out,:],
                               U=control_signal[seq], mean_predict=False, nSamples=100)
    
    pred_mean, = plot(y_pd[:,:,dim].mean(0),'b', label='prediction-mean')
    pred_var, = plot(y_pd[:,:,dim].mean(0)-y_pd[:,:,dim].std(0)*2, 'b--', label='prediction-variance')
    plot(y_pd[:,:,dim].mean(0)+y_pd[:,:,dim].std(0)*2, 'b--')
    ground_truth, = plot(data_in[seq][:,dim],'r',label='ground-truth')
    ylabel('Normalised Value')
    xlabel('Samples')
    title(model+': Dimension-'+str(dim+1))
    legend(handles=[pred_mean, pred_var, ground_truth], prop={'size':7})
    if WRITE_FIG is True:
        savefig('./images/'+model+str(dim))
#     savefig('./images/control/train_'+model+str(dim))

In [ ]:
control_signal = [np.ones((np.shape(y_train[0])[0],1)),
                  np.ones((np.shape(y_train[1])[0],1)),
                  np.ones((np.shape(y_train[2])[0],1)),
                  np.ones((np.shape(y_train[3])[0],1))]
y_test = y_test_dict['root']
pred_plot(0, 20, 20, 'root', y_test, control_signal, 0, path='./models/delta_sequence_root/')